User Question
   ↓
Embed Question
   ↓
FAISS Similarity Search
   ↓
Top-k Wikipedia Chunks
   ↓
Prompt Construction
   ↓
LLM Answer (grounded)


In [1]:
!pip install faiss-cpu sentence-transformers transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 64.3 MB/s eta 0:00:00


In [2]:
!pip install pypdf sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 12.7 MB/s eta 0:00:00


In [4]:
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer


In [13]:
def pdfreaderload(path):
  reader=PdfReader(path)
  text=""
  for page in reader.pages:
    text+=page.extract_text().rstrip()+"\n"
  return text


path="/content/RAGG.pdf"
finaltext=pdfreaderload(path)
print(finaltext)




Advanced RAG Models with Graph Structures: 
Optimizing Complex Knowledge Reasoning and Text 
Generation 
  
  
  
  
 
  
  
  
  
 
  
  
  
  
 
  
  
  
 
  
  
  
 
  
  
  
  
 
 
Abstract—This study aims to optimize the existing 
retrieval-augmented generation model (RAG) by 
introducing a graph structure to improve the performance 
of the model in dealing with complex knowledge reasoning 
tasks. The traditional RAG model has the problem of 
insufficient processing efficiency when facing complex 
graph structure information (such as knowledge graphs, 
hierarchical relationships, etc.), which affects the quality 
and consistency of the generated results. This study 
proposes a scheme to process graph structure data by 
combining graph neural network (GNN), so that the model 
can capture the complex relationship between entities, 
thereby improving the knowledge consistency and 
reasoning ability of the generated text. The experiment 
used the Natural Questions (NQ) dataset and com

In [14]:
print(len(finaltext))


27902


In [15]:
import unicodedata

# NFKD = Normalization Form Compatibility Decomposition
def normalize_unicode(text):
    return unicodedata.normalize("NFKD", text)


In [16]:
import re

def remove_extra_whitespace(text):
    # Remove multiple blank lines
    text = re.sub(r'\n\s*\n+', '\n\n', text)
    return text.strip()


In [21]:
from pypdf import PdfReader
import unicodedata
import re

def load_and_clean_pdf(path):
    reader = PdfReader(path)
    text = ""

    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"

    # Normalize unicode
    text = unicodedata.normalize("NFKD", text)

    # Remove non-printable characters
    text = ''.join(c for c in text if c.isprintable())

    # Collapse excessive blank lines
    text = re.sub(r'\n\s*\n+', '\n\n', text)

    return text.strip()

finaltext = load_and_clean_pdf("/content/RAGG.pdf")
print(finaltext)


Advanced RAG Models with Graph Structures: Optimizing Complex Knowledge Reasoning and Text Generation                                                    Abstract—This study aims to optimize the existing retrieval-augmented generation model (RAG) by introducing a graph structure to improve the performance of the model in dealing with complex knowledge reasoning tasks. The traditional RAG model has the problem of insufficient processing efficiency when facing complex graph structure information (such as knowledge graphs, hierarchical relationships, etc.), which affects the quality and consistency of the generated results. This study proposes a scheme to process graph structure data by combining graph neural network (GNN), so that the model can capture the complex relationship between entities, thereby improving the knowledge consistency and reasoning ability of the generated text. The experiment used the Natural Questions (NQ) dataset and compared it with multiple existing generation mod

In [22]:
print(finaltext.count("�"))


45


In [27]:
def remove_replacement_chars(text):
    return text.replace("�", "")
finaltext=remove_replacement_chars(finaltext)

In [28]:
print(finaltext)

Advanced RAG Models with Graph Structures: Optimizing Complex Knowledge Reasoning and Text Generation                                                    Abstract—This study aims to optimize the existing retrieval-augmented generation model (RAG) by introducing a graph structure to improve the performance of the model in dealing with complex knowledge reasoning tasks. The traditional RAG model has the problem of insufficient processing efficiency when facing complex graph structure information (such as knowledge graphs, hierarchical relationships, etc.), which affects the quality and consistency of the generated results. This study proposes a scheme to process graph structure data by combining graph neural network (GNN), so that the model can capture the complex relationship between entities, thereby improving the knowledge consistency and reasoning ability of the generated text. The experiment used the Natural Questions (NQ) dataset and compared it with multiple existing generation mod

In [31]:
print(len(finaltext))

27263


In [33]:
27263/400

68.1575

In [29]:
with open("ragg_clean.txt",'w',encoding="utf-8") as f:
  f.write(finaltext)

In [30]:
def chunk_text(text,chunk_size=500,overlap=100):
  chunks =[]
  start=0

  while start<len(finaltext):
    end = start + chunk_size
    chunk = text[start:end].strip()
    chunks.append(chunk)
    start=end-overlap

  return chunks

chunks=chunk_text(finaltext)
print(len(chunks))


69


In [36]:
from sentence_transformers import SentenceTransformer

In [37]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [38]:
embeddings=embedder.encode(
    chunks,
    convert_to_numpy=True,
    show_progress_bar=True
)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [39]:
import faiss

faiss.normalize_L2(embeddings)


In [42]:
dim=embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

In [43]:
def retrieve(query, k=3):
    # Encode query
    query_vec = embedder.encode([query], convert_to_numpy=True)

    # Normalize query (NON-OPTIONAL)
    faiss.normalize_L2(query_vec)

    # Search
    scores, indices = index.search(query_vec, k)

    return scores[0], [chunks[i] for i in indices[0]]


In [44]:
scores, results = retrieve("What is Retrieval Augmented Generation?", k=3)

for i, (score, text) in enumerate(zip(scores, results)):
    print(f"\n--- Result {i+1} | Score: {score:.4f} ---")
    print(text[:300])



--- Result 1 | Score: 0.7137 ---
round information. To this end, the retrieval -augmented generation model (RAG) came into being. By combining the retrieval system with the generation model, it can dynamically access external knowledge during the generation process, thereby greatly improving the quality and accuracy of generation. 

--- Result 2 | Score: 0.5787 ---
eneration effect. Secondly, the synergy between the retrieval module and the generation module still needs to be further optimized. Although the current RAG model can use external information to enhance the generation effect, it still has inaccurate or incoherent generation results when dealing with

--- Result 3 | Score: 0.5269 ---
range of text -generation tasks. However, in order to further improve the performance in knowledge -intensive and reasoning tasks, we combined the design of the RAG (Retrieval Augmented Generation) model to dynamically obtain relevant information from external knowledge bases through the retriev

In [45]:
import numpy as np

print(np.linalg.norm(embeddings, axis=1).min(),
      np.linalg.norm(embeddings, axis=1).max())


0.99999994 1.0000001


In [46]:
pip install anthropic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.9/397.9 kB 19.4 MB/s eta 0:00:00


In [47]:
def build_rag_prompt(context_chunks, question):
    context = "\n\n".join(context_chunks)

    return f"""
You are a factual assistant.
Answer the question using ONLY the context below.
If the answer is not present, say "I don't know".

Context:
{context}

Question:
{question}

Answer:
"""


AuthenticationError: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}, 'request_id': 'req_011CXdHpKFn1zSqq94GG8EbN'}

In [ ]:
from anthropic import Anthropic

# Initialize the client with your API key
client = Anthropic(api_key="")

def ask_claude(prompt):
    # Send the prompt to Claude via the Messages API
    response = client.messages.create(
        model="claude-opus-4-5-20251101",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=600,
        temperature=0.0
    )
    # Extract the text response
    # Depending on the SDK version, you can access the text like this:
    return response.content[0].text


# Example usage (RAG context)

question = "What is graph rag"

# Perform retrieval using your FAISS retrieval function
scores, retrieved_chunks = retrieve(question, k=3)

# Build the RAG prompt with your function
prompt = build_rag_prompt(retrieved_chunks, question)

# Make the Claude call
answer = ask_claude(prompt)

print("Answer:\n", answer)


Answer:
 Based on the context provided, **Graph RAG** is a model that combines the Retrieval-Augmented Generation (RAG) model with graph structure. It uses graph structure data (which expresses complex relationships between different objects in the form of nodes and edges) and incorporates technologies such as graph neural networks (GNNs) to better process structured knowledge and extract valuable information.

The Graph RAG model can:
- Better handle complex knowledge reasoning and generation tasks
- Process associations between structured information
- Improve the generation quality and rationality of the model

In practical applications, such as the medical field, the Graph RAG model can be used for tasks like medical record generation and diagnosis assistance, where it can better handle associations between structured information like patient history, diagnoses, and treatment plans.


# **imporved rag with metada chnaging or adding how we store stuff**

In [56]:
from pypdf import PdfReader

In [62]:
def load_pdf_with_pages(path):
  text_with_page = []
  reader = PdfReader(path)
  for i, page in enumerate(reader.pages):
    page_text = page.extract_text() or ""
    page_text = page_text.strip()
    text_with_page.append((i + 1, page_text))
    return text_with_page
pdf_pages = load_pdf_with_pages("/content/RAGG.pdf")
print("Pages loaded:", len(pdf_pages))

Pages loaded: 1


In [63]:
from pypdf import PdfReader

reader = PdfReader("/content/RAGG.pdf")
print("Total pages in PDF:", len(reader.pages))

for i, page in enumerate(reader.pages):
    txt = page.extract_text()
    print(f"Page {i+1}: extracted text length = {len(txt) if txt else 0}")


Total pages in PDF: 5
Page 1: extracted text length = 4809
Page 2: extracted text length = 5640
Page 3: extracted text length = 5629
Page 4: extracted text length = 5855
Page 5: extracted text length = 5982


In [64]:
from pypdf import PdfReader

def load_pdf_with_pages(path):
    reader = PdfReader(path)
    pages = []

    for i, page in enumerate(reader.pages):
        # extract text; fallback to empty string
        page_text = page.extract_text() or ""
        # strip leading/trailing whitespace
        cleaned = page_text.strip()

        # Always append with page number (even if empty)
        pages.append((i + 1, cleaned))

    return pages

# load the PDF
pdf_pages = load_pdf_with_pages("/content/RAGG.pdf")

print("Total pages in loader:", len(pdf_pages))
for p, txt in pdf_pages:
    print(f"Page {p} length:", len(txt))


Total pages in loader: 5
Page 1 length: 4809
Page 2 length: 5639
Page 3 length: 5628
Page 4 length: 5854
Page 5 length: 5967


In [65]:
def chunk_with_metadata(pages, chunk_size=500, overlap=100):
    """
    pages: list of (page_number, page_text)
    returns:
      list of {"text": ..., "pages": [...]} dictionaries
    """

    chunks = []
    buffer_text = ""
    buffer_pages = set()

    for page_num, page_txt in pages:
        pos = 0

        # Slide across this page text
        while pos < len(page_txt):
            piece = page_txt[pos:pos + chunk_size]

            if piece.strip():
                buffer_text += piece
                buffer_pages.add(page_num)

            # When buffer_text is large enough, emit a chunk
            if len(buffer_text) >= chunk_size:
                chunks.append({
                    "text": buffer_text.strip(),
                    "pages": sorted(buffer_pages)
                })
                # Create overlap
                buffer_text = buffer_text[-overlap:]
                buffer_pages = {page_num}

            pos += chunk_size

    # Add any leftover text as a final chunk
    if buffer_text.strip():
        chunks.append({
            "text": buffer_text.strip(),
            "pages": sorted(buffer_pages)
        })

    return chunks


In [66]:
pdf_pages = load_pdf_with_pages("/content/RAGG.pdf")
chunks_with_meta = chunk_with_metadata(pdf_pages)

print("Total chunks:", len(chunks_with_meta))
print(chunks_with_meta[0])


Total chunks: 55
{'text': 'Advanced RAG Models with Graph Structures: \nOptimizing Complex Knowledge Reasoning and Text \nGeneration \n  \n  \n  \n  \n \n  \n  \n  \n  \n \n  \n  \n  \n  \n \n  \n  \n  \n \n  \n  \n  \n \n  \n  \n  \n  \n \n \nAbstract—This study aims to optimize the existing \nretrieval-augmented generation model (RAG) by \nintroducing a graph structure to improve the performance \nof the model in dealing with complex knowledge reasoning \ntasks. The traditional RAG model has the problem of \ninsufficient processing efficiency when facing c', 'pages': [1]}


In [67]:
import re
import unicodedata

def clean_text(text):
    # Normalize unicode (remove ligatures, smart quotes)
    text = unicodedata.normalize("NFKD", text)

    # Replace newlines with spaces
    text = text.replace("\n", " ")

    # Collapse multiple spaces into one
    text = re.sub(r"\s+", " ", text).strip()

    return text


In [68]:
def chunk_with_clean_metadata(pages, chunk_size=500, overlap=100):
    chunks = []
    buffer_text = ""
    buffer_pages = set()

    for page_num, page_txt in pages:
        # Clean the page text first
        page = clean_text(page_txt)

        pos = 0
        while pos < len(page):
            piece = page[pos:pos + chunk_size]

            if piece.strip():
                buffer_text += piece + " "
                buffer_pages.add(page_num)

            # When enough text for a chunk
            if len(buffer_text) >= chunk_size:
                chunks.append({
                    "text": buffer_text.strip(),
                    "pages": sorted(buffer_pages)
                })
                buffer_text = buffer_text[-overlap:]
                buffer_pages = {page_num}

            pos += chunk_size

    # leftover
    if buffer_text.strip():
        chunks.append({
            "text": buffer_text.strip(),
            "pages": sorted(buffer_pages)
        })

    return chunks


In [80]:
pdf_pages = load_pdf_with_pages("/content/RAGG.pdf")
clean_chunks = chunk_with_clean_metadata(pdf_pages)

print("Total clean chunks:", len(clean_chunks))
# print(clean_chunks)


Total clean chunks: 54


In [71]:
from sentence_transformers import SentenceTransformer
import faiss

# 1) Initialize the embedder
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# 2) Prepare list of chunk texts
texts = [chunk["text"] for chunk in chunks_with_meta]

# 3) Generate embeddings
embeddings = embedder.encode(
    texts,
    convert_to_numpy=True,
    show_progress_bar=True
)

# 4) Normalize embeddings so dot product = cosine similarity
faiss.normalize_L2(embeddings)

# 5) Create a FAISS index for Inner Product
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

print("FAISS index built. Total vectors:", index.ntotal)


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

FAISS index built. Total vectors: 55


In [81]:
def retrieve_with_meta(query, k=3):
    # Embed the query
    q_emb = embedder.encode([query], convert_to_numpy=True)

    # Normalize the query (MANDATORY for cosine)
    faiss.normalize_L2(q_emb)

    # Search FAISS
    scores, indices = index.search(q_emb, k)

    results = []
    for score, idx in zip(scores[0], indices[0]):
        chunk = chunks_with_meta[idx]
        results.append({
            "score": float(score),
            "text": chunk["text"],
            "pages": chunk["pages"]
        })
    return results


In [82]:
def build_rag_prompt_with_meta(retrieved, question):
    context_str = ""
    for i, item in enumerate(retrieved):
        pages = ", ".join(map(str, item["pages"]))
        context_str += f"[Chunk {i+1} | Pages {pages}]\n{item['text']}\n\n"

    return f"""
You are a factual assistant. Answer using ONLY the context below.
If the answer isn’t in the context, say "I don't know".

Context:
{context_str}

Question:
{question}

Answer:
"""


In [ ]:
from anthropic import Anthropic

client = Anthropic(api_key="")

def ask_claude(prompt):
    response = client.messages.create(
        model="claude-opus-4-5-20251101",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=600,
        temperature=0.0,
    )
    return response.content[0].text


In [85]:
question = "What is Retrieval Augmented Generation?"

# 1) Retrieve
retrieved = retrieve_with_meta(question, k=3)
print("Retrieved chunks:", len(retrieved),retrieved)
# 2) Build prompt
prompt = build_rag_prompt_with_meta(retrieved, question)

# 3) Ask Claude
answer = ask_claude(prompt)

print("Answer:\n", answer)


Retrieved chunks: 3 [{'score': 0.7139130234718323, 'text': 'or learning. Although these models can generate \nrelatively coherent text, their generation ability is still limited \nwhen facing external knowledge or complex background \ninformation. To this end, the retrieval -augmented generation \nmodel (RAG) came into being. By combining the retrieval \nsystem with the generation model, it can dynamically access \nexternal knowledge during the generation process, thereby \ngreatly improving the quality and accuracy of generation. \nEspecially in tasks that require a rich knowledge background or \ncomplex reasoning, the RAG model shows stronger advantages \nthan', 'pages': [1]}, {'score': 0.5119883418083191, 'text': 'on capabilities and can handle a \nwide range of text -generation tasks. However, in order to \nfurther improve the performance in knowledge -intensive and \nreasoning tasks, we combined the design of the RAG (Retrieval \nAugmented Generation) model to dynamically obtain r